In [1]:
# importar bibliotecas

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint


In [43]:
def funcao_reacao_parcial (z, t):
    
    L, OD = z # Definir as variaveis
    
    # Constantes da reação

    V = 25.6 * (10**6) #m3
    H = 5.6 #m
    Os = 9.0 #mg/L
    v_s = 1 #m/d
    L_F4 = 1.5 #mg/L
    L_TD4 = 1.7 #mg/L
    L_TE10 = 1.3 #mg/L
    OD_F4 = 8 #mg/L
    OD_TD4 = 8 #mg/L
    OD_TE10 = 8 #mg/L
    kd = 1.5 #d-1
    ka = 0.3 #d-1
    k_sed = v_s/H
    k_ra = 0.05 #d-1
    k_n = 0.1 #d-1
    k_g = 0.15 #d-1
    r_ca = 50 #gC/mgChla
    r_oc = 2.67 #gO/gC
    r_on = 4.2 #gO/gN
    L_n = 0.157 #mg/L
    alpha = 0.0128 #mg/L
    L_part = 0.5 #adimensional
    SOD = 1 #gO2/m2
    pa = r_oc*r_ca*k_g*alpha #mgOD/L

    # Balanço hidrico

    Q_F4 = 2.0  * 86400 #* 1000 #m3/d
    Q_TD4 = 0.3 * 86400 #* 1000#m3/d
    Q_TE10 = 0.15  * 86400# * 1000#m3/d
       
    bal_hid = Q_F4 + Q_TD4 + Q_TE10  
   
    #Cargas externas

    carga_L = L_F4*Q_F4 + L_TD4*Q_TD4 + L_TE10*Q_TE10
    carga_OD = OD_F4*Q_F4 + OD_TD4*Q_TD4 + OD_TE10*Q_TE10
   
    # Definir as funções
    
    dLdt = - kd*L + r_ca*r_oc*k_ra*alpha - k_sed*(1-L_part)*L - (L/V)*bal_hid + carga_L/V   
    
    dODdt = -kd*L + ka*(Os-OD) - r_on*k_n*L_n - r_ca*r_oc*k_ra*alpha - r_ca*r_oc*k_ra*alpha - (SOD/H) + pa  - (OD/V)*bal_hid + carga_OD/V  

    return dLdt, dODdt

# Condições iniciais

L0_1 = 1.5 #mg/L
OD0_1 = 5.8 #mg/L
z0_1 = L0_1, OD0_1


# Passo no tempo

t = np.linspace(0,100,300)

In [ ]:
# Integrar as equações com odeint

funcao_teste = odeint(funcao_reacao_parcial, z0_1, t)
L1, OD1 = funcao_teste.T

# Plotar o resultado

fig1 = plt.figure(facecolor = 'w')
ax1 = fig1.add_subplot(111, facecolor='#dddddd', axisbelow=True)
ax1.plot (t, L1, 'b', label = 'DBO')
ax1.plot (t, OD1, 'r', label = 'OD')
ax1.set_xlabel ('Tempo')
ax1.set_ylabel ('Concentração DBO/OD (mg/L)')
ax1.yaxis.set_tick_params(length=0)
ax1.xaxis.set_tick_params(length=0)
ax1.grid(b=True, which='major', c='w', lw=2, ls='-')
legend1 = ax1.legend()
legend1.get_frame().set_alpha(0.5)
for spine in ('top', 'right', 'bottom', 'left'):
    ax1.spines[spine].set_visible(False)
plt.show()

In [ ]:
def funcao_reacao_variando (z, t):
    
    L, OD = z # Definir as variaveis
    
    # Constantes da reação

    V = 25.6 * (10**6) #m3
    H = 5.6 #m
    Os = 9.0 #mg/L
    v_s = 1 #m/d
    L_F4 = 1.5 #mg/L
    L_TD4 = 1.7 #mg/L
    L_TE10 = 1.3 #mg/L
    OD_F4 = 8 #mg/L
    OD_TD4 = 8 #mg/L
    OD_TE10 = 8 #mg/L
    kd = 1.5 #d-1
    ka = 0.3 #d-1
    k_sed = v_s/H
    k_ra = 0.05 #d-1
    k_n = 0.1 #d-1
    k_g = 0.15 #d-1
    r_ca = 50 #gC/mgChla
    r_oc = 2.67 #gO/gC
    r_on = 4.2 #gO/gN
    L_n = 0.157 #mg/L
    alpha = 0.0128 #mg/L
    L_part = 0.5 #adimensional
    SOD = 1 #gO2/m2
    pa = r_oc*r_ca*k_g*alpha #mgOD/L

    # Balanço hidrico

    Q_F4 = 2.0  * 86400 #* 1000 #m3/d
    Q_TD4 = 0.3 * 86400 #* 1000#m3/d
    Q_TE10 = 0.15  * 86400# * 1000#m3/d
       
    bal_hid = Q_F4 + Q_TD4 + Q_TE10  
   
    #Cargas externas

    carga_L = L_F4*Q_F4 + L_TD4*Q_TD4 + L_TE10*Q_TE10
    carga_OD = OD_F4*Q_F4 + OD_TD4*Q_TD4 + OD_TE10*Q_TE10
   
    # Definir as funções
    
    dLdt = - kd*L + r_ca*r_oc*k_ra*alpha - k_sed*(1-L_part)*L - (L/V)*bal_hid + carga_L/V   
    
    dODdt = -kd*L + ka*(Os-OD) - r_on*k_n*L_n - r_ca*r_oc*k_ra*alpha - r_ca*r_oc*k_ra*alpha - (SOD/H) + pa  - (OD/V)*bal_hid + carga_OD/V  

    return dLdt, dODdt

# Condições iniciais

L0_2 = 1.5 #mg/L
OD0_2 = 5.8 #mg/L
z0_2 = L0_2, OD0_2


# Passo no tempo

t = np.linspace(0,100,300)

In [ ]:
# Integrar as equações com odeint

funcao_teste = odeint(funcao_reacao_variando, z0_1, t)
L2, OD2 = funcao_teste.T

# Plotar o resultado

fig2 = plt.figure(facecolor = 'w')
ax2 = fig2.add_subplot(111, facecolor='#dddddd', axisbelow=True)
ax2.plot (t, L2, 'b', label = 'DBO')
ax2.plot (t, OD2, 'r', label = 'OD')
ax2.set_xlabel ('Tempo')
ax2.set_ylabel ('Concentração DBO/OD (mg/L)')
ax2.yaxis.set_tick_params(length=0)
ax2.xaxis.set_tick_params(length=0)
ax2.grid(b=True, which='major', c='w', lw=2, ls='-')
legend2 = ax2.legend()
legend2.get_frame().set_alpha(0.5)
for spine in ('top', 'right', 'bottom', 'left'):
    ax2.spines[spine].set_visible(False)
plt.show()